### Een parser-generator voor de wordgrammar

De parser-generator voor de ETCBC-wordgrammar-bestanden is gegenereerd met Yapps2 (zie de website: http://theory.stanford.edu/~amitp/yapps/ en https://github.com/smurfix/yapps) met het grammar-bestand `wgr.g`, wat resulteert in de grammar-parser `wgr.py`. Dat script is afhankelijk van de runtime-module van Yapps, hier meegeleverd als het bestand `yapps-runtime.py`.

Wat we eigenlijk willen is niet het parsen van de `wordgrammar`-bestanden, wat `wgr.py` doet, maar het parsen van de morphologische analyse van de ETCBC-database. Dat gebeurt in `wrdgrm.py`. Dat is, behalve van `wgr.py` en `yapps-runtime.py`, nog afhankelijk van de modules `alphabet.py` en `lexicon.py` (**TODO**: dat moet makkelijker kunnen). Tenslotte zijn de databestanden vereist: het alfabet, het lexicon en de wordgrammar zelf.

In [2]:
# eerst modules importeren
import os, wrdgrm

In [3]:
filepath = lambda rel_path: os.path.realpath(os.path.join(os.getcwd(), rel_path))

In [4]:
# bestandslocaties
alphabet_file = filepath('../../data/wordgrammar/alphabet')
lexicon_file = filepath('../../data/blc/syrlex')
word_grammar_file = filepath('../../data/blc/syrwgr')
an_file = filepath('../../data/blc/Laws.an')

In [5]:
# dan kan de parser worden geïnitialiseerd
w = wrdgrm.wrdgrm(word_grammar_file, lexicon_file, alphabet_file)

De functie `analyze_word()` geeft een analyse als een tuple met drie waarden: `morphemes`, `functions` en `lex`.

De eerste, `morphemes`, bevat een tuple met alle gevonden morfemen, elk als een tuple met drie strings: de paradigmatische vorm (zoals die in het lexicon staat), de oppervlaktevorm (zoals die in de tekst staat), en de geannoteerde vorm met morfologische codering.

De tweede waarde, `functions`, bevat de grammaticale functies van het woord, zoals die in de `wordgrammar` gedefinieerd zijn: `ps: "person"`, `nu: "number"`, `gn: "gender"`, `ls: "lexical set"`, `sp: "part of speech"`, `st: "state"`, `vo: "voice"`, `vs: "verbal stem"`, `vt: "verbal tense"`. Een veld met de waarde `False` geeft aan dat deze functie niet van toepassing is op dit woord, een veld met waarde `None` geeft aan dat de waarde niet is vastgesteld.

De derde waarde, `lex`, bevat het lemma zoals dat in het lexicon staat, met als eerste het woord-id, en vervolgens de annotaties. Behalve standaard-waarden voor de grammaticale functies bevat het lexicon een `gl`-veld voor ieder woord (gloss), en soms een `de`-veld (derived form). (In één resp. twee gevallen komen ook de velden `cs` en `ln` voor, waarvan de betekenis mij niet duidelijk is.)

In [6]:
# voorbeeld
w.analyze_word('>TR/&WT=~>')

((('lex', ('>TR', '>TR', '>TR')),
  ('nme', ('T=', 'WT', '&WT=')),
  ('emf', ('>', '>', '>'))),
 (('vt', False),
  ('vs', False),
  ('ps', False),
  ('sp', 'subs'),
  ('nu', 'pl'),
  ('gn', 'm'),
  ('st', 'emph')),
 ('17789', (('sp', 'subs'), ('gn', 'm'), ('gl', 'place, region'))))

In [11]:
# toon alle beschikbare data uit de an-file:
def print_anfile(an_file):
    with open(an_file) as f:
        for line in f:
            verse, s, a = line.split()
            yield ' '.join((verse, s, a))
            for e in a.split('-'):
                morphemes, functions, lex = w.analyze_word(e)
                yield f'\t{morphemes}'
                yield f'\t{functions}'
                yield f'\t{lex}'

for i, line in enumerate(print_anfile(an_file)):
    print(line)
    if i == 20:
        print ('...')
        break

0,1 TWB TWB
	(('lex', ('TWB', 'TWB', 'TWB')),)
	(('nu', False), ('gn', False), ('st', False), ('vt', False), ('vs', False), ('ps', False), ('sp', 'advb'))
	('11299', (('sp', 'advb'), ('gl', 'again, back')))
0,1 KTB> KTB=/~>
	(('lex', ('KTB=', 'KTB', 'KTB=')), ('nme', ('', '', '')), ('emf', ('>', '>', '>')))
	(('vt', False), ('vs', False), ('ps', False), ('sp', 'subs'), ('nu', None), ('gn', 'm'), ('st', 'emph'))
	('8929', (('sp', 'subs'), ('gn', 'm'), ('gl', 'writing, book')))
0,1 DNM"WS> D-NMWS/(J~>
	(('lex', ('D', 'D', 'D')),)
	(('nu', False), ('gn', False), ('st', False), ('vt', False), ('vs', False), ('ps', False), ('sp', 'prep'), ('ls', 'pcon'))
	('7789', (('sp', 'prep'), ('ls', 'pcon'), ('gl', '(relative)')))
	(('lex', ('NMWS', 'NMWS', 'NMWS')), ('nme', ('J', '', '(J')), ('emf', ('>', '>', '>')))
	(('vt', False), ('vs', False), ('ps', False), ('sp', 'subs'), ('nu', 'pl'), ('gn', 'm'), ('st', 'emph'))
	('2063', (('sp', 'subs'), ('gn', 'm'), ('gl', 'nome, prefecture, law, custom, us

In [9]:
# toon output als in dmp-file:
def dump_anfile(an_file):
    with open(an_file) as f:
        for line in f:
            verse, s, a = line.split()
            heading = f'BLC {verse}'
            for e in a.split('-'):
                morphemes, functions, lex = w.analyze_word(e)
                surface_form = ''.join((m[1][1] for m in morphemes if m[0] != 'vpm'))
                lexeme = dict(morphemes)['lex'][0]
                affixes = [m for m in morphemes if m[0] != 'lex'] # TODO affix may not be the right term?
                affix_str = ('-' if not affixes else
                    ','.join((f'{e[0]}="{e[1][0]}"' if e[0] != 'vpm' else f'{e[0]}={e[1][0]}' for e in affixes)))
                func_str = ','.join(('+'+fn if fv is None else fn+'='+fv for fn, fv in functions if fv != False))

                yield '\t'.join((heading, e, surface_form, lexeme, affix_str, func_str))

for i, line in enumerate(dump_anfile(an_file)):
    print(line)
    if i == 20:
        print ('...')
        break

BLC 0,1	TWB	TWB	TWB	-	sp=advb
BLC 0,1	KTB=/~>	KTB>	KTB=	nme="",emf=">"	sp=subs,+nu,gn=m,st=emph
BLC 0,1	D	D	D	-	sp=prep,ls=pcon
BLC 0,1	NMWS/(J~>	NMWS>	NMWS	nme="J",emf=">"	sp=subs,nu=pl,gn=m,st=emph
BLC 0,1	D	D	D	-	sp=prep,ls=pcon
BLC 0,1	>TR/&WT=~>	>TRWT>	>TR	nme="T=",emf=">"	sp=subs,nu=pl,gn=m,st=emph
BLC 1,1	MN	MN	MN	-	sp=prep
BLC 1,1	QDM	QDM	QDM	-	sp=prep
BLC 1,1	JWM/T=~>	JWMT>	JWM	nme="T=",emf=">"	sp=subs,nu=pl,gn=m,st=emph
BLC 1,1	<L=[/JN	<LJN	<L=	vbe="",nme="JN"	sp=verb,nu=pl,gn=m,st=abs,vo=act,vs=pe,vt=ptc
BLC 1,1	HWJ[N	HWJN	HWJ	vbe="N"	nu=pl,sp=verb,vo=act,vs=pe,vt=pf,ps=first,ls=vbex
BLC 1,1	L	L	L	-	sp=prep
BLC 1,1	!M!S<R=[/	MS<R	S<R=	pfm="M",vbe="",nme=""	sp=verb,+st,vo=act,vs=pe,vt=inf
BLC 1,1	L	L	L	-	sp=prep
BLC 1,1	CMCGRM/	CMCGRM	CMCGRM	nme=""	sp=subs,+nu,+gn,st=abs,ls=prop
BLC 1,1	>X/&W	>XW	>X	nme=""	sp=subs,+nu,+gn,+st
BLC 1,1	N	N	N	-	nu=pl,ps=first,sp=pron,ls=pers
BLC 1,1	W	W	W	-	sp=conj
BLC 1,1	>T(J&>[	>T>	>TJ	vbe=""	nu=sg,gn=m,sp=verb,vo=act,vs=pe,vt=pf,ps=third
BLC

Om te controleren dat de output correct is heb ik bovenstaande output vergeleken met de bestaande .dmp-bestanden. Omdat de volgorde van de waarden willekeurig lijkt te zijn - of in ieder geval niet in alle gevallen gelijk - moeten alle waarden gesorteerd worden voor ze vergeleken kunnen worden, een eenvoudige diff volstaat niet. Onderstaand script bevestigt dat bovenstaande output, op de volgorde na, een exacte weergave is van de bestaande .dmp-bestanden:

In [8]:
dmp_file = filepath('../../data/blc/Laws.dmp')
dmp_gen = dump_anfile(an_file)

with open(dmp_file) as f_dmp:
    for line1, line2 in zip(f_dmp, dmp_gen):
        for f1, f2 in zip(line1.strip().split('\t'), line2.split('\t')):
            f1s, f2s = (','.join(sorted(f.split(','))) for f in (f1,f2))
            if f1s != f2s:
                print(f'{line1}!=\n{line2}')